In [1]:
import pandas as pd
import requests
from lxml import html
from __future__ import division
from lxml.etree import tostring
import time
import random

In [41]:
def createProxy():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get('https://hidemy.name/ru/proxy-list/?country=RU&type=hs#list', headers=headers)
    tree = html.fromstring(r.content)
    
    proxy = {}
    ip_list = tree.xpath('//table[@class="proxy__t"]/tbody/tr/td[1]/text()')
    port_list = tree.xpath('//table[@class="proxy__t"]/tbody/tr/td[2]/text()')
    type_list = tree.xpath('//table[@class="proxy__t"]/tbody/tr/td[5]/text()')
    for i in range(0, len(ip_list)):
        proxy[i+1] = type_list[i]+';'+str(ip_list[i])+':'+str(port_list[i])
    
    print(len(proxy))
    return proxy

In [42]:
global proxy 
proxy = createProxy()

57


In [39]:
def prePage(url):
    rnd = random.randint(1, len(proxy)+1)
    proxies = {
      proxy[rnd][:proxy[rnd].find(';')]: proxy[rnd][proxy[rnd].find(';')+1:]
        #'https':'185.8.5.219:443'
    }
    time.sleep(2)
    page = requests.post(url, proxies=proxies)
    tree = html.fromstring(page.content.replace('&nbsp;', '').replace('\n', '').replace('\t','').replace('\r','').decode('utf-8'))
    #print('<h1>GOOD</h1>')
    return tree

In [40]:
t = prePage('https://zoon.ru/msk/beauty/')

query_param = '''need[]:items
need[]:points
search_query_form:1
locations-metro[]:Авиамоторная
locations-metro[]:Арбатская
districts[]:zamoskvoreche
page:1'''

In [165]:
def parseListing(metro = [], district = []):
    data = pd.DataFrame()
    a = []
    href = []
    stations = []
    rating = []
    address = []
    photo_count = []
    review_count = []
    tags = []
    photo_urls = []
    lon = []
    lat = []
    ID = []
    
   
    if(len(metro) == 0) and (len(district) == 0):
        num_page = 1
        try:
            page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&page=1')
        except:
            print 'CAPTCHA_BAN_HOME_LISTING'
        
        #while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0) and (page.cssselect('body')[0].text_content().find('captcha') == -1):
        while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0):
            try:
                page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&page={}'.format(num_page))
            except:
                print 'END_OF_PAGES_IN_HOME_LISTING'
                break

            if(1 != True):
                print 'CRZY WRLD'
            else:
            
                for one_div in page.cssselect('li.service-item.pd-m.js-results-item.service-features  '):

                    #Lon, Lat, ZoonID
                    lon.append(one_div.attrib.get('data-lon'))
                    lat.append(one_div.attrib.get('data-lat'))
                    ID.append(one_div.attrib.get('data-id'))

                    #PhotoUrls
                    for bar in one_div.cssselect('div.bar.js-photo-bar'):
                        photo_urls.append(bar.attrib.get('data-photos'))

                    #title and url
                    for anchor in one_div.cssselect('div.service-description div.H3 a'):
                        anch = anchor.text \
                                .replace('\n', '') \
                                .replace('\t', '')
                        if(anch != ''):
                            a.append(anch)
                            href.append(anchor.attrib.get("href"))

                    #rating
                    for rate in one_div.cssselect('div.stars-view.stars-view-star13.clearfix'):
                        solo_rating = 0
                        for star in rate.cssselect('span.star-item span'):
                            solo_rating += int(star.attrib.get('style')[star.attrib.get('style').find('width: ')+len('width: '):star.attrib.get('style').find('%;')])/100
                        rating.append(solo_rating)

                    #Tags
                    tag = []
                    for one_tag in one_div.cssselect('address.invisible-links')[0]:

                        if(one_tag.tag == 'a'):
                            tag.append(one_tag.text_content().encode('utf-8'))
                        elif(one_tag.tag == 'span'):
                            s = tostring(one_tag, encoding='utf-8')
                            s = s[s.find('<span>')+6:s.find('</span>')]
                            if(s != 'Салоны красоты и СПА'):
                                tag.append(s)
                        #print(tag)
                    tags.append(','.join(tag))
                            #print(tostring(one_tag, encoding='utf-8'))

                    #address
                    drop = one_div.cssselect('address.invisible-links i')[1]
                    one = one_div.cssselect('address.invisible-links')[1].remove(drop)
                    s = one_div.cssselect('address.invisible-links')[1].text_content()
                    s_1 = s.split(',')[0][:2].encode('utf-8')
                    if(s_1 == 'м.'):
                        stations.append(s.split(',')[0][2:])
                        address.append(','.join(s.split(',')[1:]))
                    else:
                        stations.append('-')
                        address.append(s)
                    
                    #PhotoCount
                    for count in one_div.cssselect('i.s-icons-service-photos'):
                        c = tostring(count, encoding='utf-8')
                        c = c[c.find('/>')+2:]
                        photo_count.append(c)

                    #ReviewCount    
                    for one_review_count in one_div.cssselect('div.pull-left.gray'):
                        review_count.append(int(one_review_count.text[:one_review_count.text.find(' ')]))

                print 'Success HOME_Listing!', 'Count: ',len(a), ', Pages: ',num_page
                num_page += 1
                
        data['ZoonID'] = ID
        data['Title'] = a
        data['URL'] = href 
        print len(stations), len(address), len(rating), len(review_count), len(photo_count), len(tags), len(photo_urls), len(lat), len(lon)
        data['Metro'] = stations
        data['Address'] = address
        data['Rating'] = rating
        data['ReviewCount'] = review_count
        data['PhotoCount'] = photo_count
        data['Tags'] = tags
        data['PhotoUrls'] = photo_urls
        data['Latitude'] = lat
        data['Longtitude'] = lon
        
        return data
    
    elif(len(metro) != 0):
        for one_metro in metro:
            num_page = 1
            try:
                page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&locations-metro[]={}&page=1'.format(one_metro.encode('utf-8')))
            except:
                print 'CAPTCHA_BAN_HOME_LISTING'
        
            
            #while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0) and (page.cssselect('body')[0].text_content().find('captcha') == -1):
            while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0):
                try:
                    page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&locations-metro[]={0}&page={1}'.format(one_metro.encode('utf-8'), num_page))
                except:
                    print 'END_OF_PAGES_IN_METRO_LISTING'
                    break
    
                if(1 != True):
                    print 'CRZY WRLD_METRO'
                else:
                    for one_div in page.cssselect('li.service-item.pd-m.js-results-item.service-features  '):

                        #Lon, Lat, ZoonID
                        lon.append(one_div.attrib.get('data-lon'))
                        lat.append(one_div.attrib.get('data-lat'))
                        ID.append(one_div.attrib.get('data-id'))

                        #PhotoUrls
                        for bar in one_div.cssselect('div.bar.js-photo-bar'):
                            photo_urls.append(bar.attrib.get('data-photos'))

                        #title and url
                        for anchor in one_div.cssselect('div.service-description div.H3 a'):
                            anch = anchor.text \
                                    .replace('\n', '') \
                                    .replace('\t', '')
                            if(anch != ''):
                                a.append(anch)
                                href.append(anchor.attrib.get("href"))

                        #rating
                        for rate in one_div.cssselect('div.stars-view.stars-view-star13.clearfix'):
                            solo_rating = 0
                            for star in rate.cssselect('span.star-item span'):
                                solo_rating += int(star.attrib.get('style')[star.attrib.get('style').find('width: ')+len('width: '):star.attrib.get('style').find('%;')])/100
                            rating.append(solo_rating)

                        #Tags
                        tag = []
                        for one_tag in one_div.cssselect('address.invisible-links')[0]:

                            if(one_tag.tag == 'a'):
                                tag.append(one_tag.text_content().encode('utf-8'))
                            elif(one_tag.tag == 'span'):
                                s = tostring(one_tag, encoding='utf-8')
                                s = s[s.find('<span>')+6:s.find('</span>')]
                                if(s != 'Салоны красоты и СПА'):
                                    tag.append(s)
                            #print(tag)
                        tags.append(','.join(tag))
                                #print(tostring(one_tag, encoding='utf-8'))

                        #address
                        drop = one_div.cssselect('address.invisible-links i')[1]
                        one = one_div.cssselect('address.invisible-links')[1].remove(drop)
                        s = one_div.cssselect('address.invisible-links')[1].text_content()
                        s_1 = s.split(',')[0][:2].encode('utf-8')
                        if(s_1 == 'м.'):
                            stations.append(s.split(',')[0][2:])
                            address.append(','.join(s.split(',')[1:]))
                        else:
                            stations.append('-')
                            address.append(s)

                        #PhotoCount
                        for count in one_div.cssselect('i.s-icons-service-photos'):
                            c = tostring(count, encoding='utf-8')
                            c = c[c.find('/>')+2:]
                            photo_count.append(c)

                        #ReviewCount    
                        for one_review_count in one_div.cssselect('div.pull-left.gray'):
                            review_count.append(int(one_review_count.text[:one_review_count.text.find(' ')]))

                    print 'Success Metro_%s_Listing!' % (one_metro), 'Count: ',len(a), ', Pages: ',num_page
                    num_page += 1
                    
        data['ZoonID'] = ID
        data['Title'] = a
        data['URL'] = href 
        print len(stations), len(address), len(rating), len(review_count), len(photo_count), len(tags), len(photo_urls), len(lat), len(lon)
        data['Metro'] = stations
        data['Address'] = address
        data['Rating'] = rating
        data['ReviewCount'] = review_count
        data['PhotoCount'] = photo_count
        data['Tags'] = tags
        data['PhotoUrls'] = photo_urls
        data['Latitude'] = lat
        data['Longtitude'] = lon
        
        return data
    
    elif(len(district) != 0):
        for one_district in district:
            num_page = 1
            page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&districts[]={}&page=1'.format(one_district))

            #while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0) and (page.cssselect('body')[0].text_content().find('captcha') == -1):
            while(len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))>0):
                page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&districts[]={0}&page={1}'.format(one_district, num_page))
                
                #if(page.cssselect('body')[0].text_content().find('captcha') != -1):
                    #print 'CAPTCHA_BAN | PAGE_NUM = ', num_page, ' DISTRICT: ', one_district
                    #break
                if(1 != True):
                    print 'CRZY_WRLD_DISTRICT'
                else:
                    for one_div in page.cssselect('li.service-item.pd-m.js-results-item.service-features  '):

                        #Lon, Lat, ZoonID
                        lon.append(one_div.attrib.get('data-lon'))
                        lat.append(one_div.attrib.get('data-lat'))
                        ID.append(one_div.attrib.get('data-id'))

                        #PhotoUrls
                        for bar in one_div.cssselect('div.bar.js-photo-bar'):
                            photo_urls.append(bar.attrib.get('data-photos'))

                        #title and url
                        for anchor in one_div.cssselect('div.service-description div.H3 a'):
                            anch = anchor.text \
                                    .replace('\n', '') \
                                    .replace('\t', '')
                            if(anch != ''):
                                a.append(anch)
                                href.append(anchor.attrib.get("href"))

                        #metro    
                        #station = []
                        #for metro in one_div.cssselect('address.invisible-links')[1].cssselect('a'):
                            #stations.append(metro.text[2:])
                            #print(metro.text)
                        '''if(len(station)>1):
                            stations.append(station[len(station)-1][2:])
                        elif(len(station)==1):
                            stations.append(station[0][2:])
                        else:
                            stations.append('-')'''

                        #rating
                        for rate in one_div.cssselect('div.stars-view.stars-view-star13.clearfix'):
                            solo_rating = 0
                            for star in rate.cssselect('span.star-item span'):
                                solo_rating += int(star.attrib.get('style')[star.attrib.get('style').find('width: ')+len('width: '):star.attrib.get('style').find('%;')])/100
                            rating.append(solo_rating)

                        #Tags
                        tag = []
                        for one_tag in one_div.cssselect('address.invisible-links')[0]:

                            if(one_tag.tag == 'a'):
                                tag.append(one_tag.text_content())
                            elif(one_tag.tag == 'span'):
                                s = tostring(one_tag, encoding='utf-8')
                                s = s[s.find('<span>')+6:s.find('</span>')]
                                if(s != 'Салоны красоты и СПА'):
                                    tag.append(s)
                            #print(tag)
                        tags.append(tag)
                                #print(tostring(one_tag, encoding='utf-8'))

                        #address
                        '''try:
                            adrs = ''
                            url = href[len(href)-1]
                            #print(url)
                            check_page = prePage(url)
                            adrs += check_page.cssselect('address.iblock')[0].text

                            for j in check_page.cssselect('address.iblock a'):
                                adrs += j.text+', '


                            print(adrs[:len(adrs)-2])
                        except:
                            pass'''

                        for one_address in one_div.cssselect('address.invisible-links')[1]:
                            #st = tostring(one_address.remove(one_address.cssselect('a')[0]), encoding='utf-8')
                            #st = st[st.find('</a>,')+5:]
                            if(one_address.tag != 'i'):
                                #st = one_address.drop_tag('a')
                                #s = one_address.remove(st)
                                stations.append(one_address.text[2:])
                                s = tostring(one_address, encoding='utf-8')
                                s = s[s.find('</a>,')+5:]
                                address.append(s)
                            #if(tostring(one_address)[tostring(one_address).find('</a>,')+5:tostring(one_address).find('<i')] != ''):
                                #address.append(st)
                            #pass

                        #PhotoCount
                        for count in one_div.cssselect('i.s-icons-service-photos'):
                            c = tostring(count, encoding='utf-8')
                            c = c[c.find('/>')+2:]
                            photo_count.append(c)

                        #ReviewCount    
                        for one_review_count in one_div.cssselect('div.pull-left.gray'):
                            review_count.append(int(one_review_count.text[:one_review_count.text.find(' ')]))


                    #print(len(a), len(href), tags)

                    print 'Success Distrcit_%s_Listing!' % (one_district), 'Count: ',len(a), ', Pages: ',num_page
                    num_page += 1
    else:
        print 'STOP THAT, MAN!!!'
                
        
    #DATA_SAVE        
    '''data['ZoonID'] = ID
    data['Title'] = a
    data['URL'] = href 
    data['Metro'] = stations
    data['Address'] = address
    data['Rating'] = rating
    data['ReviewCount'] = review_count
    data['PhotoCount'] = photo_count
    for i in range(0, 3):
        data['Tags_'+str(i)] = [x[i] for x in tags]
    data['PhotoUrls'] = photo_urls
    #data['LastReview'] = 
    data['Latitude'] = lat
    data['Longtitude'] = lon
        
    return data'''

In [149]:
zoon = parseListing()

Success HOME_Listing! Count:  34 , Pages:  1
Success HOME_Listing! Count:  63 , Pages:  2
Success HOME_Listing! Count:  93 , Pages:  3
Success HOME_Listing! Count:  123 , Pages:  4
Success HOME_Listing! Count:  153 , Pages:  5
Success HOME_Listing! Count:  183 , Pages:  6
Success HOME_Listing! Count:  213 , Pages:  7
Success HOME_Listing! Count:  243 , Pages:  8
END_OF_PAGES_IN_HOME_LISTING
243 243 243 243 243 243 243 243 243


In [157]:
zoon = zoon.drop_duplicates()

In [163]:
metro = metroList()

SUCCESS | METRO = 236


In [ ]:
zoon_metro = parseListing(metro = metro)

Success Metro_Авиамоторная_Listing! Count:  30 , Pages:  1
Success Metro_Авиамоторная_Listing! Count:  51 , Pages:  2
Success Metro_Авиамоторная_Listing! Count:  51 , Pages:  3
Success Metro_Автозаводская (Московское центральное кольцо)_Listing! Count:  81 , Pages:  1
Success Metro_Автозаводская (Московское центральное кольцо)_Listing! Count:  110 , Pages:  2
Success Metro_Автозаводская (Московское центральное кольцо)_Listing! Count:  121 , Pages:  3
Success Metro_Автозаводская (Московское центральное кольцо)_Listing! Count:  121 , Pages:  4
Success Metro_Автозаводская_Listing! Count:  151 , Pages:  1
Success Metro_Автозаводская_Listing! Count:  180 , Pages:  2
Success Metro_Автозаводская_Listing! Count:  199 , Pages:  3
Success Metro_Автозаводская_Listing! Count:  199 , Pages:  4
Success Metro_Академическая_Listing! Count:  229 , Pages:  1
Success Metro_Академическая_Listing! Count:  259 , Pages:  2
Success Metro_Академическая_Listing! Count:  289 , Pages:  3
Success Metro_Академическ

In [ ]:
zoon_metro

In [153]:
zoon.to_csv('zoon_home_listing.csv', encoding='utf-8')

# Parse MetroList and DistrictList

In [162]:
def metroList():
    page = prePage('https://zoon.ru/layer/metro/?city=msk&LayerSelect=undefined')
    metro = []
    
    try:
        #if(page.cssselect('body')[0].text_content().find('captcha') != -1):
            #print 'CAPTCHA_BAN | METRO'
        #else:
        for value in page.cssselect('ul.metro-stations li input'):
            metro.append(value.attrib.get('value'))
        if(len(metro)>0):
            print 'SUCCESS | METRO =',len(metro) 
            return metro
        else:
            print('CAPTCHA_BAN')
    except:
        print('CAPTCHA_BAN')

In [5]:
def districtList():
    page = prePage('https://zoon.ru/layer/district/?city=msk&LayerSelect=undefined')
    districts = {}
    
    try:
        #if(page.cssselect('body')[0].text_content().find('captcha') != -1):
            #print 'CAPTCHA_BAN | DISTRICTS'
        #else:
        for value in page.cssselect('div.select-block.block-fluid ul li input'):
            districts[value.attrib.get('value')] = value.attrib.get('data-caption')
        if(len(districts.keys())>0):
            print 'SUCCESS | DISTRICTS =',len(districts.keys()) 
            return districts
        else:
            print('CAPTCHA_BAN')
    except:
        print('CAPTCHA_BAN')

# MEGA_PARSE

In [6]:
def logZOON():
    #try:
    #metro = metroList()
    #district = districtList()
    #districts = district.keys()

    log = pd.DataFrame()
        
    log_1 = parseListing()
        #log_2 = parseListing(metro = metro)
        #log_3 = parseListing(district = districts)
    
    log = log_1.copy()
        #log = log.append(log_2)
        #log = log.append(log_3)
        
    log = log.drop_duplicates()
        
    return log
    #except:
        #print('OOOOOPS!')

In [7]:
log = logZOON()

0 0 0 0 0 0 0 0 0


In [115]:
log

,ZoonID,Title,URL,Metro,Address,Rating,ReviewCount,PhotoCount
0,4f145febf89c60510c000cc7,Тату 3000,https://zoon.ru/cardbanner/click/?ticket=NTk0N...,None,None,4.4,20,52
1,5955db1bb453033cba618bd6,Follow Beauty,https://zoon.ru/cardbanner/click/?ticket=NTk1Y...,None,None,4.7,29,24
2,58c76e856be6a167648b4575,EPI CENTER,https://zoon.ru/cardbanner/click/?ticket=NThlN...,None,None,4.2,5,13
3,52c0233b40c0886b7c8bdf37,КОММО,https://zoon.ru/cardbanner/click/?ticket=NTk2Y...,None,None,4.6,49,17
4,5742d5f7e828628d0f8b4587,Beauty bar Wosk&Go,https://zoon.ru/msk/beauty/salon_krasoty_beaut...,None,None,4.6,49,19
5,51a70069a0f3027f1e000016,Тиффани,https://zoon.ru/msk/beauty/salon_krasoty_tiffa...,None,None,4.2,37,29
6,55f8e2a3774836ea208b4571,РеФорма,https://zoon.ru/msk/beauty/salon_krasoty_refor...,None,None,4.2,46,41
7,519345a0a0f3029b6700000c,Искусство стиля,https://zoon.ru/msk/beauty/salon_krasoty_iskus...,None,None,4.1,90,43
8,58dc9de9e90875fd1f8b4568,SHIC&LAC,https://zoon.ru/msk/beauty/studiya_shiclac_v_s...,None,None,4.5,46,25
9,545c40bd8ab12d3e5b8b45ad,МедлайН-Сервис,https://zoon.ru/msk/medical/meditsinskij_tsent...,None,None,4.2,122,20


In [89]:
log

,ZoonID,Title,URL
0,5955db1bb453033cba618bd6,Follow Beauty,https://zoon.ru/cardbanner/click/?ticket=NTk1Y...
1,51a3140ba0f3026c50000020,Сальвадоре,https://zoon.ru/cardbanner/click/?ticket=NThlM...
2,52203250a0f3020d23000025,ЭпилСити,https://zoon.ru/cardbanner/click/?ticket=NTk0M...
3,5924dbf9b666ef163e59fed4,Barbarella,https://zoon.ru/cardbanner/click/?ticket=NTk0M...
4,5742d5f7e828628d0f8b4587,Beauty bar Wosk&Go,https://zoon.ru/msk/beauty/salon_krasoty_beaut...
5,51a70069a0f3027f1e000016,Тиффани,https://zoon.ru/msk/beauty/salon_krasoty_tiffa...
6,55f8e2a3774836ea208b4571,РеФорма,https://zoon.ru/msk/beauty/salon_krasoty_refor...
7,519345a0a0f3029b6700000c,Искусство стиля,https://zoon.ru/msk/beauty/salon_krasoty_iskus...
8,58dc9de9e90875fd1f8b4568,SHIC&LAC,https://zoon.ru/msk/beauty/studiya_shiclac_v_s...
9,545c40bd8ab12d3e5b8b45ad,МедлайН-Сервис,https://zoon.ru/msk/medical/meditsinskij_tsent...


In [53]:
page = prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&page=6')

In [114]:
#a = []
def check(page):
    k = 0
    for one_div in page.cssselect('li.service-item.pd-m.js-results-item.service-features  '):

        drop = one_div.cssselect('address.invisible-links i')[1]
        one = one_div.cssselect('address.invisible-links')[1].remove(drop)
        s = one_div.cssselect('address.invisible-links')[1].text_content()
        s_1 = s.split(',')[0][:2].encode('utf-8')
        if(s_1 == 'м.'):
            #print s_1
            k+=1
        #for one_address in one_div.cssselect('address.invisible-links')[1]:
            #print(one_address.text_content())
            #k+=1
            #print(one_address)
            #s = tostring(one_address, encoding='utf-8')
            #k+=1
            #print s
            #if(one_address.text != None):
                #s = tostring(one_address, encoding='utf-8')
                #s = s[s.find('</a>,')+5:]
                #print a,'-',one_address.text[2:],':', s
                #k += 1
            #else:
                #print a,'| - : -'
                
    print 'F:',k,'All:',len(page.cssselect('li.service-item.pd-m.js-results-item.service-features  '))

In [125]:
check( prePage('https://zoon.ru/msk/beauty/?action=list&type=service&need[]=items&page=10'))

XMLSyntaxError: line 1: htmlParseEntityRef: expecting ';' (line 1)